<a href="https://colab.research.google.com/github/RCortez25/Scientific-Machine-Learning/blob/main/Differential_equations/BrusselatorPDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Brusselator PDE

$$
\begin{align}
\partial_tu &= 1+vu^2-4.4u+\alpha(\partial_{xx}u+\partial_{yy}u)+f(x,y,t) \\
\partial_tv &= 3.4u-vu^2+\alpha(∂_{xx}v+∂_{yy}v)
\end{align}
$$

where

$$
f(x,y,t)=
\begin{cases}
5, & (x-0.3)^2+(y-0.6)^2\leq(0.1)^2~\text{and}~t\geq1.1 \\
0, & \text{otherwise}
\end{cases}
$$

Initial conditions

$$
\begin{align}
u(x,y,0) &=22(y(1-y))^{3/2} \\
v(x,y,0) &=27(x(1-x))^{3/2}
\end{align}
$$

Periodic boundary condition
$$
\begin{align}
u(x+1,y,t) &=u(x,y,t) \\
u(x,y+1,t) &=u(x,y,t)
\end{align}
$$

Also

$$
t\in[0,11.5]
$$

# Code

In [ ]:
# A
@parameters x y t
@variables u(..) v(..)

# B
Dt = Differential(t)
Dx = Differential(x)
Dy = Differential(y)
Dxx = Differential(x)^2
Dyy = Differential(y)^2

# C
∇²(u) = Dxx(u) + Dyy(u)



*   **A**: Define the independent variables `x, y, t`, and the dependent variables `u, v`.
*   **B**: Define the operators to be used, in this case

$$
\partial_t,\partial_x,\partial_{y},\partial_{xx},\partial_{yy}
$$

*   **C**: Definition of the Laplacian operator as well

$$\partial_{xx}+\partial_{yy}$$



In [ ]:
# A
brusselator_f(x, y, t) = (((x-0.3)^2 + (y-0.6)^2) <= 0.1^2) * (t >= 1.1) * 5

# B
x_min = y_min = t_min = 0.0
x_max = y_max = 1.0
t_max = 11.5

# C
α = 10.0

# D
u0(x, y, t) = 22 * (y * (1 - y))^(3/2)
v0(x, y, t) = 27 * (x * (1 - x))^(3/2)



*   **A**: The PDE contains a function $f(x,y,t)$ as defined above. This is expressed in code here.
*   **B**: Definition of the min and max values for each independent variable, to be used in the boundary conditions later. In this case

$$
\begin{align}
x &\in[0,1] \\
y &\in[0,1] \\
t &\in[0,11.5]
\end{align}
$$

*   **C**: Choosing a value for the constant $\alpha=10$

*   **D**: Definition of the initial conditions, int his case

$$
\begin{align}
u_0\equiv u(x,y,0) &= 22(y(1-y))^{3/2} \\
v_0\equiv v(x,y,0) &= 27(x(1-x))^{3/2} \\
\end{align}
$$


In [ ]:
# A
equations = [
    Dt(u(x,y,t)) ~ 1 + v(x,y,t)*u(x,y,t)^2 - 4.4*u(x,y,t) + α*∇²(u(x,y,t)) + brusselator_f(x,y,t),
    Dt(v(x,y,t)) ~ 3.4*u(x,y,t) - v(x,y,t)*u(x,y,t)^2 + α*∇²(v(x,y,t))
]

# B
domains = [
    x ∈ Interval(x_min, x_max),
    y ∈ Interval(y_min, y_max),
    t ∈ Interval(t_min, t_max)
]



*   **A**: Definition of the two PDEs using the defined operators

$$
\begin{align}
\partial_tu &= 1+vu^2-4.4u+\alpha(\partial_{xx}u+\partial_{yy}u)+f(x,y,t) \\
\partial_tv &= 3.4u-vu^2+\alpha(∂_{xx}v+∂_{yy}v)
\end{align}
$$

*   **B**: Definition of the domains of each independent variable using the values from above



In [ ]:
# A
boundary_conditions = [
    u(x,y,0) ~ u0(x,y,0),
    u(0,y,t) ~ u(1,y,t),
    u(x,0,t) ~ u(x,1,t),

    v(x,y,0) ~ v0(x,y,0),
    v(0,y,t) ~ v(1,y,t),
    v(x,0,t) ~ v(x,1,t)
]




*   **A**: Defining the periodic boundary conditions, in this case one has

$$
\begin{align}
u(x,y,0) &= u_0 \\
u(0,y,t) &= u(1,y,t) \\
u(x,0,t) &= u(x,1,t)
\end{align}
$$

That is, the value of $u$ is the same at $x=0$ and at $x=1$, $\forall t$. The same applies for $u$ at the values of $y=0$ and $y=1$.

The same boundary conditions apply for $v$

$$
\begin{align}
v(x,y,0) &= v_0 \\
v(0,y,t) &= v(1,y,t) \\
v(x,0,t) &= v(x,1,t)
\end{align}
$$

In [ ]:
# A
@named pde_system = PDESystem(equations, boundary_conditions, domains, [x,y,t], [u(x,y,t), v(x,y,t)])



*   **A**: Definition of the PDE system of equations using the `PDESystem` constructor. One passes it the equations, the boundary conditions, the domains of the independent variables, the independent variables, and the dependent variables.

In [ ]:
# A
N = 32
order = 2

# B
discretization = MOLFiniteDifference([x=>N, y=>N], t, approx_order=order)



*   **A**: Setting the parameters to pass to the MOL method
*   **B**: One tells the method to discretize `x, y`, into `N=32` grid points, but leave `t` continous. This creates a square of $x,y$ points, which will be a 32$\times$32 grid for the domain $[0,1]$, so each square will be $\Delta x=\Delta y=1/32$ and to use it an approximation order of 2, that is, a second-order finite difference scheme for the derivatives. So, this basically makes a grid.

Recall that the Method of Lines transforms the PDE system into a large system of ODEs in time.



In [ ]:
# A
problem = discretize(pde_system, discretization)

# B
solution = solve(problem, TRBDF2(), saveat=0.1)



*   **A**: Take the PDE system and applyt the discretization rule to it. The returns an ODEProblem object. This writes down equations for every grid cell. In this case, $32\times32=1024$ ODEs for $u$ and $1024$ for $v$.
*   **B**: Solve the problem using the `TRBDF2` algorithm, which is a second-order implicit solver, good for stiff problems. The solution is saved every 0.1 units. This integrates the ODE forward in time until `t_max=11.5`. With this, one watches patterns evolve.



In [ ]:
# A
x_grid = solution[x]
y_grid = solution[y]
t_grid = solution[t]

# B
u_solution = solution[u(x,y,t)]
v_solution = solution[v(x,y,t)]



*   **A**: Retrieves the points on the grid ($x$ and $y$) and the points $t$ at which the solution was saved.
*   **B**: Retrieves the solution of $u$ and $v$ as arrays with the computed values at each point of the grid at every saved time.


In [ ]:
# A
anim_u = @animate for k in 1:length(t_grid)
    heatmap(u_solution[2:end, 2:end, k], title="$(t_grid[k])")
end

# B
gif(anim_u, "Brusselator_u.gif", fps = 8)

# C
anim_v = @animate for k in 1:length(t_grid)
    heatmap(v_solution[2:end, 2:end, k], title="$(t_grid[k])")
end
gif(anim_v, "Brusselator_v.gif", fps = 8)



*   **A**: For the variable $u$, for each saved time step, plots one heatmap frame and collects it into an animation
*   **B**: Combines all collected frames of `anim_u` into an animation and saves it as a `.gif` file with 8 frames per second.
*   **C**: The same for the variable $v$

